Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[!, :log_gdp] = log.(dcc[!, :rgdppc_2000])
dcc[!, :cont_africa] = Array{Float64}(convert(Array{Int}, dcc[!, :cont_africa]))

170-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0

First 5 rows with data

In [4]:
first(dcc[!, [:rugged, :cont_africa, :log_gdp]], 5)

struct m_8_1_model{TY <: AbstractVector, TX <: AbstractMatrix}
    "Observations."
    y::TY
    "Covariates"
    X::TX
end

Make the type callable with the parameters *as a single argument*.

In [5]:
function (problem::m_8_1_model)(θ)
    @unpack y, X, = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    ll = 0.0
    ll += logpdf(Normal(0, 100), X[1]) # a = X[1]
    ll += logpdf(Normal(0, 10), X[2]) # bR = X[2]
    ll += logpdf(Normal(0, 10), X[3]) # bA = X[3]
    ll += logpdf(Normal(0, 10), X[4]) # bAR = X[4]
    ll += logpdf(TDist(1.0), σ)
    ll += loglikelihood(Normal(0, σ), y .- X*β)
    ll
end

Instantiate the model with data and inits.

In [6]:
N = size(dcc, 1)
X = hcat(ones(N), dcc[!, :rugged], dcc[!, :cont_africa], dcc[!, :rugged].*dcc[!, :cont_africa]);
y = convert(Vector{Float64}, dcc[!, :log_gdp])
p = m_8_1_model(y, X);
p((β = [1.0, 2.0, 1.0, 2.0], σ = 1.0))

-2770.7279383721293

Write a function to return properly dimensioned transformation.

In [7]:
problem_transformation(p::m_8_1_model) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [8]:
P = TransformedLogDensity(problem_transformation(p), p)
#∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
∇P = ADgradient(:ForwardDiff, P);
LogDensityProblems.stresstest(P)

0-element Array{Array{Float64,1},1}

Tune and sample.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.00035 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00035 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00034 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00043 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00021 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00022 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00022 s/step ...done
MCMC (1000 steps)
0.00023 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [9.203988901873045, -0.219860203643808, -2.075430760082664, 0.420402788443498], σ = 0.9157344046660246)    
 (β = [9.316528894112563, -0.2255563617086166, -1.629198137271803, 0.3019032921160433], σ = 0.9052720655860147)  
 (β = [9.061169305830644, -0.06388810288551833, -1.8516102478868897, 0.3400770725786228], σ = 0.9489398840697613)
 (β = [9.10546438363916, -0.13235218193936177, -2.2235651687908833, 0.5574265126584673], σ = 0.9332921040781665) 
 (β = [9.440493392895132, -0.30564567753601063, -2.151650060654471, 0.42314277483815144], σ = 0.9561543071923729)

Extract the parameter posterior means: `β`,

In [11]:
posterior_β = mean(first, posterior)

4-element Array{Float64,1}:
  9.223824733533423  
 -0.2021843646109359 
 -1.9461992924842437 
  0.39131964225599364

then `σ`:

In [12]:
posterior_σ = mean(last, posterior)

0.9493491574825778

Effective sample sizes (of untransformed draws)

In [13]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)

1×5 Array{Float64,2}:
 1000.0  1000.0  1000.0  1000.0  1000.0

NUTS-specific statistics

In [14]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.36 0.85 0.94 0.99 1.0
  termination: AdjacentTurn => 18% DoubledTurn => 82%
  depth: 1 => 1% 2 => 42% 3 => 57% 4 => 0%


Result rethinking

In [15]:
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Summary

In [16]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [9.223824733533423, -0.2021843646109359, -1.9461992924842437, 0.39131964225599364]
 0.9493491574825778                                                                 

End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*